In [1]:
import json
import gzip
import os
from datetime import timedelta, date
import pandas as pd

In [2]:
from paper.config import *

by_dir

'/mnt/storage2/Git/jooble-ir-2018-paper/Data/by'

In [22]:
by_dir = "/mnt/storage2/Data/by"
list_json = []

start_date = date(2017, 3, 1)
# end_date = date(2017, 1, 30)
end_date = date(2017, 3, 3)

date_range = pd.date_range(start_date, end_date)

#looping through all days from 2017_01_1 to 2017_01_30 with step = 1 day
for single_date in date_range:
    date_ = single_date.strftime("%Y-%m-%d")
        
    #!!!!CHANGE PATH!!!!!!
    file_job = os.path.join(by_dir, 'by_job_{}.json.gz'.format(date_))
    file_job_text = os.path.join(by_dir, 'by_job_text_{}.json.gz'.format(date_))

    #reading two files simuntaneously
    document_job, document_job_text = [],[]
    
    with gzip.open(file_job, "rb") as f1, gzip.open(file_job_text, "rb") as f2: 
        for line1, line2 in zip(f1, f2):
            vacation_job = json.loads(line1)
            vacation_job_text = json.loads(line2)
            document_job.append(vacation_job)
            document_job_text.append(vacation_job_text)

        #creating a list of dictionaries
        for i in range(len(document_job)):
            for j in range(len(document_job_text)):
                if document_job[i]['id'] == document_job_text[j]['id_job']:
                    data = {}
                    data['id'] = document_job[i]['id']
                    data['lang_text'] = document_job[i]['lang_text']
                    data['lang_title'] = document_job[i]['lang_title']
                    data['company_name'] = document_job[i]['company_name']
                    data['title'] = document_job[i]['title']
                    data['title_normalized'] = document_job[i]['title_normalized']
                    data['salary_val1'] = document_job[i]['salary_val1']
                    data['salary_val2'] = document_job[i]['salary_val2']
                    data['company_name'] = document_job[i]['company_name']
                    data['url'] = document_job[i]['url']
                    
                    data['text'] = document_job_text[j]['text']
                    list_json.append(data)
                    
print(len(list_json))

101775


In [17]:
! cat /mnt/storage2/Git/jooble-ir-2018-paper/Data/by/by_job_2017-03-02.json.gz
! Data/by/


cat: /mnt/storage2/Git/jooble-ir-2018-paper/Data/by/by_job_2017-03-02.json.gz: No such file or directory


In [23]:
from paper.analyze import norm_stem_lang
from paper.analyze import normalize


@timed
def stemm_d(list_json):
    # stemmed = [norm_stem_lang(doc['title']+ ' ' + doc['text']).split() for doc in list_json]
    stemmed = []
    for doc in list_json:
        try:
            stemmed.append(norm_stem_lang(doc['title']+ ' ' + doc['text']).split())
        except:
            pass
        
stemm_d(list_json)

stemm_d: 1996.782083272934


In [17]:
# stemmed[25:30]

In [5]:
list_json[24]

{'company_name': '',
 'id': '-4161554798960957173',
 'lang_text': 'ru',
 'lang_title': 'ru',
 'salary_val1': '',
 'salary_val2': '',
 'text': 'Требуется SEO специалист знающий специфику продвижения в США, требуется подвинуть сайт по 2-3 ВЧ запросам по одному штату.',
 'title': 'SEO-оптимизатор. Продвижение сайта в США (удаленно)',
 'title_normalized': 'seo оптимизатор продвижение сайта в сша удаленно',
 'url': 'https://www.fl.ru/projects/3085129/prodvijenie-sayta-v-ssha-.html?utm_source=joobleby&utm_medium=cpc&utm_campaign=jooble'}

In [7]:
# list_json[:2]

In [21]:
%load_ext autoreload
from paper import index
from paper.utils import timed

@timed
def update_timed(list_json, stemmed):
    index.update_index(list_json, stemmed)

# index.delete_all()
update_timed(list_json, stemmed);

update_timed: 33.62377214431763


In [14]:
from redis import Redis


tokens = ["inv+ "+tok for tok in ['водител', 'такс']]
r = Redis()
r.sinter(*tokens)

{b'10046',
 b'1047',
 b'1196',
 b'122',
 b'127',
 b'1427',
 b'1700',
 b'1724',
 b'2082',
 b'2164',
 b'2262',
 b'239',
 b'2507',
 b'2557',
 b'2628',
 b'2677',
 b'2696',
 b'2874',
 b'292',
 b'2998',
 b'3050',
 b'3252',
 b'3309',
 b'3461',
 b'3623',
 b'3766',
 b'3961',
 b'4058',
 b'4472',
 b'4880',
 b'4893',
 b'5230',
 b'5278',
 b'5364',
 b'5404',
 b'5676',
 b'5882',
 b'5969',
 b'6131',
 b'6144',
 b'6541',
 b'6571',
 b'6823',
 b'7058',
 b'7154',
 b'7217',
 b'7761',
 b'781',
 b'7885',
 b'8',
 b'8035',
 b'8516',
 b'866',
 b'8992',
 b'9050',
 b'9368',
 b'9509',
 b'966',
 b'9695'}

In [36]:
! pip install redis==2.10.6

    100% |████████████████████████████████| 71kB 1.1MB/s ta 0:00:011
  Found existing installation: redis 3.0.0.post1
    Uninstalling redis-3.0.0.post1:
      Successfully uninstalled redis-3.0.0.post1


In [ ]:
from paper.index import get_docs

get_docs([1,2])

## Make Index update via `updater` service

In [ ]:
from paper.utils import post_request as post
from paper.index import delete_all

delete_all()

update_json = {'docs': list_json[100:200]}

post(UPDATER_PORT, UPDATER_PATH, update_json)